In [ ]:
import pandas as pd
from googlesearch import search 
from urllib.request import urlretrieve
from urllib.request import Request, urlopen 
import requests  
import urllib.request

import PyPDF2
from PyPDF2 import PdfFileReader, PdfFileWriter
import re
import tabula
import os
import warnings
warnings.filterwarnings("ignore")


from io import BytesIO

import re

## Helper Functions

In [ ]:
import re
def getWordsFromURL(url):
    return re.compile(r'[\:/?=\-&]+',re.UNICODE).split(url)


def check_keyword(keywords):
    for keyword in keywords:
        if ".pdf" in keyword:
            return True
    return False

## Task 2 - Extract Tables

In [ ]:
def get_tables(url):
    class AppURLopener(urllib.request.FancyURLopener):
        version = "Mozilla/5.0"
        
        
    opener = AppURLopener()
    remoteFile = opener.open(url).read()
    memoryFile = BytesIO(remoteFile)
    
    print("Scanning url "+str(url))
    dfs = tabula.read_pdf(memoryFile, pages="all")
    pdfs = []
    for df in dfs:
        s = df.to_string()
        matches = ["water", "sewer", "capital","operational","utlities","p.u","distribution","meter"]
        if any(x in s.lower() for x in matches):
            pdfs.append(df)
            
    print("Found "+str(len(pdfs))+" tables")
    return pdfs

## Task 1 - Extract PDFs with PWS name

In [ ]:
def main(name):
    num_pdfs = 0
    missed = 0
    name = name.lower()
    query = name + " municipal utility water budget fiscal year 2020 pdf"
    urls = []
    print('-------------------------')
    print('Searching for query : '+str(query))
    print('--------------------------')
    for j in search(query, tld="com", num=10, stop=10, pause=2): 
        urls.append(j)
    for url in urls:
        words = getWordsFromURL(url)
        keywords = [x.lower() for x in words]
        if check_keyword(keywords):
            num_pdfs += 1
            try:
                budget_tables = get_tables(url)
            except:
                print('URL failed to download - '+str(url))

        else:
            print('No query match')
            missed += 1

        break
    return budget_tables

## Run the below cell for full pipeline

In [ ]:
pws_name = "City of Chicago"
dfs = main(pws_name)

In [ ]:
dfs[0]